<a href="https://colab.research.google.com/github/nick01as/Gazer/blob/main/Stargazing_Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opendatasets
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import opendatasets as od
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from math import radians, cos, sin, asin, sqrt
from google.colab import files

# Haversince Distance Formula (https://medium.com/analytics-vidhya/finding-nearest-pair-of-latitude-and-longitude-match-using-python-ce50d62af546)

def dist(lat1, long1, lat2, long2):
    lat1, long1, lat2, long2 = map(radians, [lat1, long1, lat2, long2])
    # haversine formula 
    dlon = long2 - long1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km

# Import LM Data
LM2021 = pd.read_csv("GaN2021New.csv")
LM2021 = LM2021.drop(columns = ['Elevation(m)','LocalTime','UTDate','UTTime','ID','ObsType','ObsID','SQMReading','SQMSerial','Constellation','SkyComment','LocationComment','Unnamed: 18'])

LM2020 = pd.read_csv("GaN2020New.csv")
LM2020 = LM2020.drop(columns = ['Elevation(m)','LocalTime','UTDate','UTTime','ID','ObsType','ObsID','SQMReading','SQMSerial','Constellation','SkyComment','LocationComment','Unnamed: 18'])

LM2019 = pd.read_csv("GaN2019New.csv")
LM2019 = LM2019.drop(columns = ['Elevation(m)','LocalTime','UTDate','UTTime','ID','ObsType','ObsID','SQMReading','SQMSerial','Constellation','SkyComment','LocationComment','Unnamed: 18'])

LM2018 = pd.read_csv("GaN2018New.csv")
LM2018 = LM2018.drop(columns = ['Elevation(m)','LocalTime','UTDate','UTTime','ID','ObsType','ObsID','SQMReading','SQMSerial','Constellation','SkyComment','LocationComment'])

LM2017 = pd.read_csv("GaN2017New.csv")
LM2017 = LM2017.drop(columns = ['Elevation(m)','LocalTime','UTDate','UTTime','ID','ObsType','ObsID','SQMReading','SQMSerial','Constellation','SkyComment','LocationComment'])

LM2016 = pd.read_csv("GaN2016New.csv")
LM2016 = LM2016.drop(columns = ['Elevation(m)','LocalTime','UTDate','UTTime','ID','ObsType','ObsID','SQMReading','SQMSerial','Constellation','SkyComment','LocationComment'])

LM2015 = pd.read_csv("GaN2015New.csv")
LM2015 = LM2015.drop(columns = ['Elevation(m)','LocalTime','UTDate','UTTime','ID','ObsType','ObsID','SQMReading','SQMSerial','Constellation','SkyComment','LocationComment'])

LM2014 = pd.read_csv("GaN2014New.csv")
LM2014 = LM2014.drop(columns = ['Elevation(m)','LocalTime','UTDate','UTTime','ID','ObsType','ObsID','SQMReading','SQMSerial','Constellation','SkyComment','LocationComment'])

# Import AQData

AQData = pd.read_csv("AQData2022.csv")
AQData = AQData.drop(columns = ['WHO Region','ISO3','PM2.5 (μg/m3)','NO2 (μg/m3)','PM25 temporal coverage (%)','PM10 temporal coverage (%)',
                                'NO2 temporal coverage (%)','Reference','Number and type of monitoring stations','Version of the database',
                                'Status','Unnamed: 15','Unnamed: 16'])

# Import City Coordinates
CityCoor = pd.read_csv("worldcities.csv")
CityCoor = CityCoor.drop(columns = ['city_ascii','iso2','iso3','admin_name','capital','population','id'])

# Sort City Locations
CityCoor = CityCoor.sort_values(['lat','lng'])
cols = list(CityCoor.columns.values)
CityCoor = CityCoor[cols[1:3] + [cols[0]] + [cols[3]]]


#Filter Light Pollution
LM2021 = LM2021.loc[(LM2021['CloudCover'] == 'clear') & (LM2021['LimitingMag'] != -999) & (LM2021['LimitingMag']) != -9999 & (LM2021['LimitingMag'] != -99999)]
LM2020 = LM2020.loc[(LM2020['CloudCover'] == 'clear') & (LM2020['LimitingMag'] != -999) & (LM2020['LimitingMag']) != -9999 & (LM2020['LimitingMag'] != -99999)]
LM2019 = LM2019.loc[(LM2019['CloudCover'] == 'clear') & (LM2019['LimitingMag'] != -999) & (LM2019['LimitingMag']) != -9999 & (LM2019['LimitingMag'] != -99999)]
LM2018 = LM2018.loc[(LM2018['CloudCover'] == 'clear') & (LM2018['LimitingMag'] != -999) & (LM2018['LimitingMag']) != -9999 & (LM2018['LimitingMag'] != -99999)]
LM2017 = LM2017.loc[(LM2017['CloudCover'] == 'clear') & (LM2017['LimitingMag'] != -999) & (LM2017['LimitingMag']) != -9999 & (LM2017['LimitingMag'] != -99999)]
LM2016 = LM2016.loc[(LM2016['CloudCover'] == 'clear') & (LM2016['LimitingMag'] != -999) & (LM2016['LimitingMag']) != -9999 & (LM2016['LimitingMag'] != -99999)]
LM2015 = LM2015.loc[(LM2015['CloudCover'] == 'clear') & (LM2015['LimitingMag'] != -999) & (LM2015['LimitingMag']) != -9999 & (LM2015['LimitingMag'] != -99999)]

# Find City

def find_city(lat,lng):
  # print (lat,lng)
  lowLB = 0
  highLB = len(CityCoor.index) 
  lowUB = 0
  highUB = len(CityCoor.index) 

  while lowLB < highLB:
    mid = lowLB + (highLB - lowLB)//2
    if CityCoor.iloc[mid,0] < lat - 0.12:
      lowLB = mid + 1;
    else:
      highLB = mid;

  while lowUB < highUB:
    mid = lowUB + (highUB - lowUB)//2
    if CityCoor.iloc[mid,0] <= lat + 0.12:
      lowUB = mid + 1;
    else:
      highUB = mid;
  
  lowInt = lowLB
  highInt = lowUB

  # print("Lower Bound Coordinate", CityCoor.iloc[lowLB,0], CityCoor.iloc[highLB,0])
  # print("Upper Bound Coordinate", CityCoor.iloc[lowUB,0], CityCoor.iloc[highUB,0])
  best = 2147483647
  city = ""
  country = ""

  for idx in range (lowLB, lowUB):
    dif = dist(lat,lng,CityCoor.iloc[idx,0],CityCoor.iloc[idx,1])
    if dif < best:
      city = CityCoor.iloc[idx,2]
      country = CityCoor.iloc[idx,3]
      best = dif

  return (city,country)

cityList = []
errorList = []
error_count = 0

LM2014
for idx in range (len(LM2014.index)):
  print (idx)
  locInfo = find_city(LM2014.iloc[idx,0],LM2014.iloc[idx,1])
  if locInfo[1] != LM2014.iloc[idx,5] and (locInfo[1][:8] != 'United S' and locInfo[1][:8] != 'Netherla' and 
                                           locInfo[1][:8] != 'Falkland' and locInfo[1][:7] != 'Czechia' and locInfo[1] != 'Mexico'
                                           and locInfo[1] != 'Macedonia' and locInfo[1] != 'Slovenia' and locInfo[1] != 'Trinidad and Tobago'
                                           and locInfo[1] != 'Belgium' and locInfo[1] != 'Vatican City' and locInfo[1] != 'Vatican' and locInfo[1] != 'Argentina' 
                                           and locInfo[1] != 'West Bank' and locInfo[1] != 'Uruguay' and locInfo[1] != 'Thailand' and locInfo[1] != 'Bolivia' 
                                           and locInfo[1] != 'France' and locInfo[1] != 'Kuwait' and locInfo[1] != 'Brazil'):
    print ("Error: Wrong Country",locInfo[1],LM2014.iloc[idx,5])
    errorList.append(1)
    error_count = error_count + 1
  else:
    errorList.append(0)

  cityList.append(locInfo[0])


print("")
print(cityList)
print(error_count)
LM2014City = LM2014.drop(columns = ['Latitude','Longitude'])
LM2014City['City'] = cityList
LM2014City['Error'] = errorList
cols = list(LM2014City.columns.values)
LM2014City = LM2014City[cols[0:3] + [cols[-2]] + [cols[-3]] + [cols[-1]]]

idx = 0
while idx < len(LM2014City.index):
  if LM2014City.iloc[idx,5] == 1:
    LM2014City = LM2014City.drop(index = LM2014City.index[idx])
  else:
    idx = idx + 1

LM2014City

LM2014City.to_csv('LM2014City.csv')
files.download('LM2014City.csv')





Streaming output truncated to the last 5000 lines.
14584
14585
14586
14587
14588
14589
14590
14591
14592
14593
14594
14595
14596
14597
14598
14599
14600
14601
14602
14603
14604
14605
14606
14607
14608
14609
14610
14611
14612
14613
14614
14615
14616
14617
14618
14619
14620
14621
14622
14623
14624
14625
14626
14627
14628
14629
14630
14631
14632
14633
14634
14635
14636
14637
14638
14639
14640
14641
14642
14643
14644
14645
14646
14647
14648
14649
14650
14651
14652
14653
14654
14655
14656
14657
14658
14659
14660
14661
14662
14663
14664
14665
14666
14667
14668
14669
14670
14671
14672
14673
14674
14675
14676
14677
14678
14679
14680
14681
14682
14683
14684
14685
14686
14687
14688
14689
14690
14691
14692
14693
14694
14695
14696
14697
14698
14699
14700
14701
14702
14703
14704
14705
14706
14707
14708
14709
14710
14711
14712
14713
14714
14715
14716
14717
14718
14719
14720
14721
14722
14723
14724
14725
14726
14727
14728
14729
14730
14731
14732
14733
14734
14735
14736
14737
14738
14739
14740
14741
1

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import opendatasets as od
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from math import radians, cos, sin, asin, sqrt
from google.colab import files

# Haversince Distance Formula (https://medium.com/analytics-vidhya/finding-nearest-pair-of-latitude-and-longitude-match-using-python-ce50d62af546)

def dist(lat1, long1, lat2, long2):
    lat1, long1, lat2, long2 = map(radians, [lat1, long1, lat2, long2])
    # haversine formula 
    dlon = long2 - long1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km

# Import LM Data
LM2021 = pd.read_csv("GaN2021New.csv")
LM2021 = LM2021.drop(columns = ['Elevation(m)','LocalTime','UTDate','UTTime','ID','ObsType','ObsID','SQMReading','SQMSerial','Constellation','SkyComment','LocationComment','Unnamed: 18'])

LM2020 = pd.read_csv("GaN2020New.csv")
LM2020 = LM2020.drop(columns = ['Elevation(m)','LocalTime','UTDate','UTTime','ID','ObsType','ObsID','SQMReading','SQMSerial','Constellation','SkyComment','LocationComment','Unnamed: 18'])

LM2019 = pd.read_csv("GaN2019New.csv")
LM2019 = LM2019.drop(columns = ['Elevation(m)','LocalTime','UTDate','UTTime','ID','ObsType','ObsID','SQMReading','SQMSerial','Constellation','SkyComment','LocationComment','Unnamed: 18'])

LM2018 = pd.read_csv("GaN2018New.csv")
LM2018 = LM2018.drop(columns = ['Elevation(m)','LocalTime','UTDate','UTTime','ID','ObsType','ObsID','SQMReading','SQMSerial','Constellation','SkyComment','LocationComment'])

LM2017 = pd.read_csv("GaN2017New.csv")
LM2017 = LM2017.drop(columns = ['Elevation(m)','LocalTime','UTDate','UTTime','ID','ObsType','ObsID','SQMReading','SQMSerial','Constellation','SkyComment','LocationComment'])

LM2016 = pd.read_csv("GaN2016New.csv")
LM2016 = LM2016.drop(columns = ['Elevation(m)','LocalTime','UTDate','UTTime','ID','ObsType','ObsID','SQMReading','SQMSerial','Constellation','SkyComment','LocationComment'])

LM2015 = pd.read_csv("GaN2015New.csv")
LM2015 = LM2015.drop(columns = ['Elevation(m)','LocalTime','UTDate','UTTime','ID','ObsType','ObsID','SQMReading','SQMSerial','Constellation','SkyComment','LocationComment'])

# Import AQData

AQData = pd.read_csv("AQData2022.csv")
AQData = AQData.drop(columns = ['WHO Region','ISO3','PM2.5 (μg/m3)','NO2 (μg/m3)','PM25 temporal coverage (%)','PM10 temporal coverage (%)',
                                'NO2 temporal coverage (%)','Reference','Number and type of monitoring stations','Version of the database',
                                'Status','Unnamed: 15','Unnamed: 16'])

# Import City Coordinates
CityCoor = pd.read_csv("worldcities.csv")
CityCoor = CityCoor.drop(columns = ['city_ascii','iso2','iso3','admin_name','capital','population','id'])

# Sort City Locations
CityCoor = CityCoor.sort_values(['lat','lng'])
cols = list(CityCoor.columns.values)
CityCoor = CityCoor[cols[1:3] + [cols[0]] + [cols[3]]]


#Filter Light Pollution
LM2021 = LM2021.loc[(LM2021['CloudCover'] == 'clear') & (LM2021['LimitingMag'] != -999) & (LM2021['LimitingMag']) != -9999 & (LM2021['LimitingMag'] != -99999)]
LM2020 = LM2020.loc[(LM2020['CloudCover'] == 'clear') & (LM2020['LimitingMag'] != -999) & (LM2020['LimitingMag']) != -9999 & (LM2020['LimitingMag'] != -99999)]
LM2019 = LM2019.loc[(LM2019['CloudCover'] == 'clear') & (LM2019['LimitingMag'] != -999) & (LM2019['LimitingMag']) != -9999 & (LM2019['LimitingMag'] != -99999)]
LM2018 = LM2018.loc[(LM2018['CloudCover'] == 'clear') & (LM2018['LimitingMag'] != -999) & (LM2018['LimitingMag']) != -9999 & (LM2018['LimitingMag'] != -99999)]
LM2017 = LM2017.loc[(LM2017['CloudCover'] == 'clear') & (LM2017['LimitingMag'] != -999) & (LM2017['LimitingMag']) != -9999 & (LM2017['LimitingMag'] != -99999)]
LM2016 = LM2016.loc[(LM2016['CloudCover'] == 'clear') & (LM2016['LimitingMag'] != -999) & (LM2016['LimitingMag']) != -9999 & (LM2016['LimitingMag'] != -99999)]
LM2015 = LM2015.loc[(LM2015['CloudCover'] == 'clear') & (LM2015['LimitingMag'] != -999) & (LM2015['LimitingMag']) != -9999 & (LM2015['LimitingMag'] != -99999)]

# Find City

def find_city(lat,lng):
  # print (lat,lng)
  lowLB = 0
  highLB = len(CityCoor.index) 
  lowUB = 0
  highUB = len(CityCoor.index) 

  while lowLB < highLB:
    mid = lowLB + (highLB - lowLB)//2
    if CityCoor.iloc[mid,0] < lat - 0.12:
      lowLB = mid + 1;
    else:
      highLB = mid;

  while lowUB < highUB:
    mid = lowUB + (highUB - lowUB)//2
    if CityCoor.iloc[mid,0] <= lat + 0.12:
      lowUB = mid + 1;
    else:
      highUB = mid;
  
  lowInt = lowLB
  highInt = lowUB

  # print("Lower Bound Coordinate", CityCoor.iloc[lowLB,0], CityCoor.iloc[highLB,0])
  # print("Upper Bound Coordinate", CityCoor.iloc[lowUB,0], CityCoor.iloc[highUB,0])
  best = 2147483647
  city = ""
  country = ""

  for idx in range (lowLB, lowUB):
    dif = dist(lat,lng,CityCoor.iloc[idx,0],CityCoor.iloc[idx,1])
    if dif < best:
      city = CityCoor.iloc[idx,2]
      country = CityCoor.iloc[idx,3]
      best = dif

  return (city,country)

cityList = []
errorList = []
error_count = 0

#--------------------------------------------------------------------------------------------------------------------------------------------
for idx in range (len(LM2020.index)):
  print (idx)
  locInfo = find_city(LM2020.iloc[idx,0],LM2020.iloc[idx,1])
  if locInfo[1] != LM2020.iloc[idx,5] and (locInfo[1][:8] != 'United S' and locInfo[1][:8] != 'Netherla' and 
                                           locInfo[1][:8] != 'Falkland' and locInfo[1][:7] != 'Czechia' and locInfo[1] != 'Mexico'
                                           and locInfo[1] != 'Macedonia' and locInfo[1] != 'Slovenia' and locInfo[1] != 'Trinidad and Tobago'
                                           and locInfo[1] != 'Belgium' and locInfo[1] != 'Vatican City' and locInfo[1] != 'Vatican' and locInfo[1] != 'Argentina' 
                                           and locInfo[1] != 'West Bank' and locInfo[1] != 'Uruguay' and locInfo[1] != 'Thailand' and locInfo[1] != 'Bolivia' 
                                           and locInfo[1] != 'France' and locInfo[1] != 'Kuwait' and locInfo[1] != 'Brazil'):
    print ("Error: Wrong Country",locInfo[1],LM2020.iloc[idx,5])
    errorList.append(1)
    error_count = error_count + 1
  else:
    errorList.append(0)

  cityList.append(locInfo[0])


print("")
print(cityList)
print(error_count)
LM2020City = LM2020.drop(columns = ['Latitude','Longitude'])
LM2020City['City'] = cityList
LM2020City['Error'] = errorList
cols = list(LM2020City.columns.values)
LM2020City = LM2020City[cols[0:3] + [cols[-2]] + [cols[-3]] + [cols[-1]]]

idx = 0
while idx < len(LM2020City.index):
  if LM2020City.iloc[idx,5] == 1:
    LM2020City = LM2020City.drop(index = LM2020City.index[idx])
  else:
    idx = idx + 1

LM2020City

LM2020City.to_csv('LM2020City.csv')
files.download('LM2020City.csv')





Streaming output truncated to the last 5000 lines.
17181
17182
17183
17184
17185
17186
17187
17188
17189
17190
17191
17192
17193
17194
17195
17196
17197
17198
17199
17200
17201
17202
17203
17204
17205
17206
17207
17208
17209
17210
17211
17212
17213
17214
17215
17216
17217
17218
17219
17220
17221
17222
17223
17224
17225
17226
17227
17228
17229
17230
17231
17232
17233
17234
17235
17236
17237
17238
17239
17240
17241
17242
17243
17244
17245
17246
17247
17248
17249
17250
17251
17252
17253
17254
17255
17256
17257
17258
17259
17260
17261
17262
17263
17264
17265
17266
17267
17268
17269
17270
17271
17272
17273
17274
17275
17276
17277
17278
17279
17280
17281
17282
17283
17284
17285
17286
17287
17288
17289
17290
17291
17292
17293
17294
17295
17296
17297
17298
17299
17300
17301
17302
17303
17304
17305
17306
17307
17308
17309
17310
17311
17312
17313
17314
17315
17316
17317
17318
17319
17320
17321
17322
17323
17324
17325
17326
17327
17328
17329
17330
17331
17332
17333
17334
17335
Error: Wrong Countr

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import opendatasets as od
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from math import radians, cos, sin, asin, sqrt
from google.colab import files

# Haversince Distance Formula (https://medium.com/analytics-vidhya/finding-nearest-pair-of-latitude-and-longitude-match-using-python-ce50d62af546)

def dist(lat1, long1, lat2, long2):
    lat1, long1, lat2, long2 = map(radians, [lat1, long1, lat2, long2])
    # haversine formula 
    dlon = long2 - long1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km

# Import LM Data
LM2021 = pd.read_csv("GaN2021New.csv")
LM2020 = pd.read_csv("GaN2020New.csv")
LM2019 = pd.read_csv("GaN2019New.csv")
LM2018 = pd.read_csv("GaN2018New.csv")
LM2017 = pd.read_csv("GaN2017New.csv")
LM2016 = pd.read_csv("GaN2016New.csv")
LM2015 = pd.read_csv("GaN2015New.csv")

# Import AQData

AQData = pd.read_csv("AQData2022.csv")
AQData = AQData.drop(columns = ['WHO Region','ISO3','PM2.5 (μg/m3)','NO2 (μg/m3)','PM25 temporal coverage (%)','PM10 temporal coverage (%)',
                                'NO2 temporal coverage (%)','Reference','Number and type of monitoring stations','Version of the database',
                                'Status','Unnamed: 15','Unnamed: 16'])

# Import City Coordinates
CityCoor = pd.read_csv("worldcities.csv")
CityCoor = CityCoor.drop(columns = ['city_ascii','iso2','iso3','admin_name','capital','population','id'])

# Sort City Locations
CityCoor = CityCoor.sort_values(['lat','lng'])
cols = list(CityCoor.columns.values)
CityCoor = CityCoor[cols[1:3] + [cols[0]] + [cols[3]]]


#Filter Light Pollution
idx = 0
while idx < len(LM2021.index):
  if (LM2021.iloc[idx,13] != 'clear') or (LM2021.iloc[idx,10] == -999) or (LM2021.iloc[idx,10] == -9999) or (LM2021.iloc[idx,10] == -99999):
    LM2021 = LM2021.drop(index = LM2021.index[idx])
  else:
    idx = idx + 1

idx = 0
while idx < len(LM2020.index):
  if (LM2020.iloc[idx,13] != 'clear') or (LM2020.iloc[idx,10] == -999) or (LM2020.iloc[idx,10] == -9999) or (LM2020.iloc[idx,10] == -99999):
    LM2020 = LM2020.drop(index = LM2020.index[idx])
  else:
    idx = idx + 1

idx = 0
while idx < len(LM2019.index):
  if (LM2019.iloc[idx,13] != 'clear') or (LM2019.iloc[idx,10] == -999) or (LM2019.iloc[idx,10] == -9999) or (LM2019.iloc[idx,10] == -99999):
    LM2019 = LM2019.drop(index = LM2019.index[idx])
  else:
    idx = idx + 1

idx = 0
while idx < len(LM2018.index):
  if (LM2018.iloc[idx,13] != 'clear') or (LM2018.iloc[idx,10] == -999) or (LM2018.iloc[idx,10] == -9999) or (LM2018.iloc[idx,10] == -99999):
    LM2018 = LM2018.drop(index = LM2018.index[idx])
  else:
    idx = idx + 1

idx = 0
while idx < len(LM2017.index):
  if (LM2017.iloc[idx,13] != 'clear') or (LM2017.iloc[idx,10] == -999) or (LM2017.iloc[idx,10] == -9999) or (LM2017.iloc[idx,10] == -99999):
    LM2017 = LM2017.drop(index = LM2017.index[idx])
  else:
    idx = idx + 1

idx = 0
while idx < len(LM2016.index):
  if (LM2016.iloc[idx,13] != 'clear') or (LM2016.iloc[idx,10] == -999) or (LM2016.iloc[idx,10] == -9999) or (LM2016.iloc[idx,10] == -99999):
    LM2016 = LM2016.drop(index = LM2016.index[idx])
  else:
    idx = idx + 1

idx = 0
while idx < len(LM2015.index):
  if (LM2015.iloc[idx,13] != 'clear') or (LM2015.iloc[idx,10] == -999) or (LM2015.iloc[idx,10] == -9999) or (LM2015.iloc[idx,10] == -99999):
    LM2015 = LM2015.drop(index = LM2015.index[idx])
  else:
    idx = idx + 1

LM2021 = LM2021.drop(columns = ['Elevation(m)','LocalTime','UTDate','UTTime','ID','ObsType','ObsID','SQMReading','SQMSerial','Constellation','SkyComment','LocationComment','Unnamed: 18'])
LM2020 = LM2020.drop(columns = ['Elevation(m)','LocalTime','UTDate','UTTime','ID','ObsType','ObsID','SQMReading','SQMSerial','Constellation','SkyComment','LocationComment','Unnamed: 18'])
LM2019 = LM2019.drop(columns = ['Elevation(m)','LocalTime','UTDate','UTTime','ID','ObsType','ObsID','SQMReading','SQMSerial','Constellation','SkyComment','LocationComment','Unnamed: 18'])
LM2018 = LM2018.drop(columns = ['Elevation(m)','LocalTime','UTDate','UTTime','ID','ObsType','ObsID','SQMReading','SQMSerial','Constellation','SkyComment','LocationComment'])
LM2017 = LM2017.drop(columns = ['Elevation(m)','LocalTime','UTDate','UTTime','ID','ObsType','ObsID','SQMReading','SQMSerial','Constellation','SkyComment','LocationComment'])
LM2016 = LM2016.drop(columns = ['Elevation(m)','LocalTime','UTDate','UTTime','ID','ObsType','ObsID','SQMReading','SQMSerial','Constellation','SkyComment','LocationComment'])
LM2015 = LM2015.drop(columns = ['Elevation(m)','LocalTime','UTDate','UTTime','ID','ObsType','ObsID','SQMReading','SQMSerial','Constellation','SkyComment','LocationComment'])

LM2015.to_csv('LM2015.csv')
files.download('LM2015.csv')
LM2016.to_csv('LM2016.csv')
files.download('LM2016.csv')
LM2017.to_csv('LM2017.csv')
files.download('LM2017.csv')
LM2018.to_csv('LM2018.csv')
files.download('LM2018.csv')
LM2019.to_csv('LM2019.csv')
files.download('LM2019.csv')
LM2020.to_csv('LM2020.csv')
files.download('LM2020.csv')
LM2021.to_csv('LM2021.csv')
files.download('LM2021.csv')

# Find City

def find_city(lat,lng):
  # print (lat,lng)
  lowLB = 0
  highLB = len(CityCoor.index) 
  lowUB = 0
  highUB = len(CityCoor.index) 

  while lowLB < highLB:
    mid = lowLB + (highLB - lowLB)//2
    if CityCoor.iloc[mid,0] < lat - 0.12:
      lowLB = mid + 1;
    else:
      highLB = mid;

  while lowUB < highUB:
    mid = lowUB + (highUB - lowUB)//2
    if CityCoor.iloc[mid,0] <= lat + 0.12:
      lowUB = mid + 1;
    else:
      highUB = mid;
  
  lowInt = lowLB
  highInt = lowUB

  # print("Lower Bound Coordinate", CityCoor.iloc[lowLB,0], CityCoor.iloc[highLB,0])
  # print("Upper Bound Coordinate", CityCoor.iloc[lowUB,0], CityCoor.iloc[highUB,0])
  best = 2147483647
  city = ""
  country = ""

  for idx in range (lowLB, lowUB):
    dif = dist(lat,lng,CityCoor.iloc[idx,0],CityCoor.iloc[idx,1])
    if dif < best:
      city = CityCoor.iloc[idx,2]
      country = CityCoor.iloc[idx,3]
      best = dif

  return (city,country)

cityList = []
errorList = []
error_count = 0

#--------------------------------------------------------------------------------------------------------------------------------------------
for idx in range (len(LM2015.index)):
  print (idx)
  locInfo = find_city(LM2015.iloc[idx,0],LM2015.iloc[idx,1])
  if locInfo[1] != LM2015.iloc[idx,5] and (locInfo[1][:8] != 'United S' and locInfo[1][:8] != 'Netherla' and 
                                           locInfo[1][:8] != 'Falkland' and locInfo[1][:7] != 'Czechia' and locInfo[1] != 'Mexico'
                                           and locInfo[1] != 'Macedonia' and locInfo[1] != 'Slovenia' and locInfo[1] != 'Trinidad and Tobago'
                                           and locInfo[1] != 'Belgium' and locInfo[1] != 'Vatican City' and locInfo[1] != 'Vatican' and locInfo[1] != 'Argentina' 
                                           and locInfo[1] != 'West Bank' and locInfo[1] != 'Uruguay' and locInfo[1] != 'Thailand' and locInfo[1] != 'Bolivia' 
                                           and locInfo[1] != 'France' and locInfo[1] != 'Kuwait' and locInfo[1] != 'Brazil' and locInfo[1] != 'Slovakia' and locInfo[1] != 'Hungary'
                                           and locInfo[1] != 'Finland' and locInfo[1] != 'Bosnia and Herzegovina'):
    print ("Error: Wrong Country",locInfo[1],LM2015.iloc[idx,5])
    errorList.append(1)
    error_count = error_count + 1
  else:
    errorList.append(0)

  cityList.append(locInfo[0])


print("")
print(cityList)
print(error_count)
LM2015City = LM2015.drop(columns = ['Latitude','Longitude'])
LM2015City['City'] = cityList
LM2015City['Error'] = errorList
cols = list(LM2015City.columns.values)
LM2015City = LM2015City[cols[0:3] + [cols[-2]] + [cols[-3]] + [cols[-1]]]

idx = 0
while idx < len(LM2015City.index):
  if LM2015City.iloc[idx,5] == 1:
    LM2015City = LM2015City.drop(index = LM2015City.index[idx])
  else:
    idx = idx + 1

LM2015City

LM2015City.to_csv('LM2015City.csv')
files.download('LM2015City.csv')





<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import requests

api_token = 'oxGvZawYkOVvhVWBQFmXWdkdgqXBZomI'  # Replace 'YOUR_API_KEY' with your actual API key
url = 'https://www.ncei.noaa.gov/cdo-web/api/v2/locations'

headers = {'token': api_token}

response = requests.get(url, headers=headers)
data = response.json()
print(data)


{'metadata': {'resultset': {'offset': 1, 'count': 38862, 'limit': 25}}, 'results': [{'mindate': '1983-01-01', 'maxdate': '2023-05-15', 'name': 'Abu Dhabi, AE', 'datacoverage': 0.9978, 'id': 'CITY:AE000001'}, {'mindate': '1944-03-01', 'maxdate': '2023-05-15', 'name': 'Ajman, AE', 'datacoverage': 0.9991, 'id': 'CITY:AE000002'}, {'mindate': '1944-03-01', 'maxdate': '2023-05-15', 'name': 'Dubai, AE', 'datacoverage': 0.9991, 'id': 'CITY:AE000003'}, {'mindate': '1944-03-01', 'maxdate': '2023-05-15', 'name': 'Sharjah, AE', 'datacoverage': 0.9991, 'id': 'CITY:AE000006'}, {'mindate': '1966-03-02', 'maxdate': '2021-08-30', 'name': 'Kabul, AF', 'datacoverage': 0.9969, 'id': 'CITY:AF000007'}, {'mindate': '1973-01-02', 'maxdate': '2020-12-31', 'name': 'Kandahar, AF', 'datacoverage': 1, 'id': 'CITY:AF000008'}, {'mindate': '1877-04-01', 'maxdate': '2023-05-15', 'name': 'Algiers, AG', 'datacoverage': 1, 'id': 'CITY:AG000001'}, {'mindate': '1909-11-01', 'maxdate': '2023-05-15', 'name': 'Annaba, AG', 'd